In [1]:
import pandas as pd
import time
import numpy as np
np.seterr(invalid='ignore')
import nbimporter
from PPP import PPP

prevGames = ['Game1', 'Game2', 'Game3']
Opp = 'Heat'
Team = 'Nuggets'
series = 'Finals'

In [2]:
players = ['braun', 'bruce', 'gordon', 'green', 'jokic', 'KCP', 'MPJ', 'murray', 'team', 'opp']

In [3]:
def load_data(players):
    
    # Establish dictionaries for new game data and overall series/season data
    new_player_data = {}
    ovr_player_data = {}
    ovr_series_player_data = {}
    team_opp = ['team', 'opp']
    
    for game in prevGames: # Loop over all games in series/season
        
        for player in players: # Loop over all players on roster
            
            if player == 'team': # Add all player data together to create team file for game

                try: # to load a team file for the game (helpful if already added team file, and/or rerunning for Opp file)
                    newplayerFile = (f'Team/{game + Opp}Team.csv')
                    newplayer = pd.read_csv(newplayerFile)
                    newplayer = newplayer.dropna()
                    newplayer = newplayer.set_index('Shot Type')
                    newplayer = newplayer.astype('int')
                    new_player_data[player] = newplayer # Add the existing new team file to the new_player_data dictionary
                except(FileNotFoundError):
                    # Load an empty df
                    newplayerFile = ('Team/empty.csv')
                    newplayer = pd.read_csv(newplayerFile)
                    newplayer = newplayer.set_index('Shot Type')

                    for key, df in new_player_data.items(): # Loop over the new_player_data and add together all player dataframes
                        if key not in team_opp: # Exclude the team and opponent dataframes
                            newplayer = newplayer.add(df)

                    new_player_data['team'] = newplayer # Add the created new team file to new_player_data dictionary
                    newplayer.to_csv(f'../{Opp}/Opp/{game + Team}Opp.csv') # Save team file to opponent folder for their opponent stats
                    newplayer.to_csv(f'Team/{game + Opp}Team.csv') # Save team file to team folder
                    
                # Load overall data for the team
                try:
                    ovrplayerFile = (f'!Data/Playoffs/Team.csv')
                    ovrplayer = pd.read_csv(ovrplayerFile)
                    ovrplayer = ovrplayer.dropna()
                    ovrplayer = ovrplayer.set_index('Shot Type')
                    ovrplayer = ovrplayer.astype('int')
                    ovr_player_data[player] = ovrplayer # Add the overall team data to the ovr_player_data dictionary
                except(FileNotFoundError):
                    ovrplayer = newplayer
                    ovr_player_data[player] = ovrplayer
                    ovrplayer.to_csv(f'!Data/Playoffs/{player.title()}.csv')
                
                # Load overall series data for the player
                try: 
                    ovr_seriesplayerFile = (f'!Data/Playoffs/{series}/{player.title() + series}.csv')
                    ovr_seriesplayer = pd.read_csv(ovr_seriesplayerFile)
                    ovr_seriesplayer = ovr_seriesplayer.dropna()
                    ovr_seriesplayer = ovr_seriesplayer.set_index('Shot Type')
                    ovr_seriesplayer = ovr_seriesplayer.astype('int')
                    ovr_series_player_data[player] = ovr_seriesplayer # Add overall player data to the ovr_player_data dictionary
                # If there is no overall series data for the player, set the new player data as the overall series data and continue to the next player    
                except(FileNotFoundError):
                    ovr_seriesplayer = newplayer
                    ovr_series_player_data[player] = ovr_seriesplayer
                    ovr_seriesplayer.to_csv(f'!Data/Playoffs/{series}/{player.title() + series}.csv')
                    continue
                    
                if newplayer.columns.equals(ovrplayer.columns) and newplayer.columns.equals(ovr_seriesplayer.columns): # Make sure the new and overall dataframes can be added without error
                    ovrplayer = ovrplayer.add(newplayer)
                    ovr_player_data[player] = ovrplayer
                    ovrplayer.to_csv(f'!Data/Playoffs/{player.title()}.csv') # Write the updated overall player file to the overall data directory
                    if game == 'Game1':
                        continue

                    ovr_seriesplayer = ovr_seriesplayer.add(newplayer)
                    ovr_series_player_data[player] = ovr_seriesplayer
                    ovr_seriesplayer.to_csv(f'!Data/Playoffs/{series}/{player.title() + series}.csv' )# Write the updated overall series player file to the overall series data directory
                else:
                    print(f"{player.title()}'s columns do not match")  
                continue

            if player == 'opp': # Load the opponent file and add the new game data to the overall opponent file if necessary

                try: # to load the new opponent file (if running this teams data first, there will be no opponent file!!)
                    newplayerFile = (f'Opp/{game + Opp}Opp.csv')
                    newplayer = pd.read_csv(newplayerFile)
                    newplayer = newplayer.dropna()
                    newplayer = newplayer.set_index('Shot Type')
                    newplayer = newplayer.astype('int')
                    new_player_data[player] = newplayer # Add the new opponent file to the new_player_data dictionary
                except(FileNotFoundError):
                    print(f'No {player.title()} file found.')
                    continue
                    
                # Load overall data for the opponent
                try:
                    ovrplayerFile = (f'!Data/Playoffs/Opp.csv')
                    ovrplayer = pd.read_csv(ovrplayerFile)
                    ovrplayer = ovrplayer.dropna()
                    ovrplayer = ovrplayer.set_index('Shot Type')
                    ovrplayer = ovrplayer.astype('int')
                    ovr_player_data[player] = ovrplayer # Add the overall opponent file to the ovr_player_data dictionary
                except(FileNotFoundError):
                    if newplayer:
                        ovrplayer = newplayer
                        ovr_player_data[player] = ovrplayer
                        ovrplayer.to_csv(f'!Data/Playoffs/{player.title()}.csv')
                    else:
                        print(f'No overall Opponent file found.')
                        continue
                        
                # Load overall series data for the player
                try: 
                    ovr_seriesplayerFile = (f'!Data/Playoffs/{series}/{player.title() + series}.csv')
                    ovr_seriesplayer = pd.read_csv(ovr_seriesplayerFile)
                    ovr_seriesplayer = ovr_seriesplayer.dropna()
                    ovr_seriesplayer = ovr_seriesplayer.set_index('Shot Type')
                    ovr_seriesplayer = ovr_seriesplayer.astype('int')
                    ovr_series_player_data[player] = ovr_seriesplayer # Add overall player data to the ovr_player_data dictionary
                # If there is no overall series data for the player, set the new player data as the overall series data and continue to the next player    
                except(FileNotFoundError):
                    ovr_seriesplayer = newplayer
                    ovr_series_player_data[player] = ovr_seriesplayer
                    ovr_seriesplayer.to_csv(f'!Data/Playoffs/{series}/{player.title() + series}.csv')
                    continue

                if newplayer.columns.equals(ovrplayer.columns) and newplayer.columns.equals(ovr_seriesplayer.columns): # Make sure the new and overall dataframes can be added without error
                    ovrplayer = ovrplayer.add(newplayer)
                    ovr_player_data[player] = ovrplayer
                    ovrplayer.to_csv(f'!Data/Playoffs/{player.title()}.csv') # Write the updated overall player file to the overall data directory
                    if game == 'Game1':
                        continue

                    ovr_seriesplayer = ovr_seriesplayer.add(newplayer)
                    ovr_series_player_data[player] = ovr_seriesplayer
                    ovr_seriesplayer.to_csv(f'!Data/Playoffs/{series}/{player.title() + series}.csv' )# Write the updated overall series player file to the overall series data directory
                else:
                    print(f"{player.title()}'s columns do not match")  
                continue
                
            ### Loading/Adding the player files (first part of the loop)###
            try:
                newplayerFile = (f'{player.title()}/{game + Opp + player.title()}.csv') # Search the team directory for the player, then concat the game/opp/player to find csv for game
                newplayer = pd.read_csv(newplayerFile)
                newplayer = newplayer.dropna()
                newplayer = newplayer.set_index('Shot Type')
                newplayer = newplayer.astype('int')
                new_player_data[player] = newplayer # Add the new player data to the new_player_data dictionary
            except(FileNotFoundError):
                print(f'No {player.title()} file found for {game}.')
                try: # to load an overall file for the player
                    ovrplayerFile = (f'!Data/Playoffs/{player.title()}.csv')
                    ovrplayer = pd.read_csv(ovrplayerFile)
                    ovrplayer = ovrplayer.dropna()
                    ovrplayer = ovrplayer.set_index('Shot Type')
                    ovrplayer = ovrplayer.astype('int')
                    ovr_player_data[player] = ovrplayer # Add overall player data to the ovr_player_data dictionary
                    continue
                except(FileNotFoundError):
                    print(f'No data for {player.title()}.')
                    continue # If no player data is found, skip and go to the next player (edit for later)
                    
                try: # to load an overall player file for the series
                    ovr_seriesplayerFile = (f'!Data/Playoffs/{player.title()}.csv')
                    ovr_seriesplayer = pd.read_csv(ovr_seriesplayerFile)
                    ovr_seriesplayer = ovr_seriesplayer.dropna()
                    ovr_seriesplayer = ovr_seriesplayer.set_index('Shot Type')
                    ovr_seriesplayer = ovr_seriesplayer.astype('int')
                    ovr_series_player_data[player] = ovr_seriesplayer # Add overall player data to the ovr_series_player_data dictionary
                    continue
                except(FileNotFoundError):
                    print(f'No {series} series data for {player.title()}.')
                    continue # If no player data is found, skip and go to the next player (edit for later)
                    
            # Load overall data for the player
            try: 
                ovrplayerFile = (f'!Data/Playoffs/{player.title()}.csv')
                ovrplayer = pd.read_csv(ovrplayerFile)
                ovrplayer = ovrplayer.dropna()
                ovrplayer = ovrplayer.set_index('Shot Type')
                ovrplayer = ovrplayer.astype('int')
                ovr_player_data[player] = ovrplayer # Add overall player data to the ovr_player_data dictionary
            # If there is no overall data for the player, set the new player data as the overall and continue to the next player    
            except(FileNotFoundError):
                ovrplayer = newplayer
                ovr_player_data[player] = ovrplayer
                ovrplayer.to_csv(f'!Data/Playoffs/{player.title()}.csv')
              
            # Load overall series data for the player
            try: 
                ovr_seriesplayerFile = (f'!Data/Playoffs/{series}/{player.title() + series}.csv')
                ovr_seriesplayer = pd.read_csv(ovr_seriesplayerFile)
                ovr_seriesplayer = ovr_seriesplayer.dropna()
                ovr_seriesplayer = ovr_seriesplayer.set_index('Shot Type')
                ovr_seriesplayer = ovr_seriesplayer.astype('int')
                ovr_series_player_data[player] = ovr_seriesplayer # Add overall player data to the ovr_player_data dictionary
            # If there is no overall series data for the player, set the new player data as the overall series data and continue to the next player    
            except(FileNotFoundError):
                ovr_seriesplayer = newplayer
                ovr_series_player_data[player] = ovr_seriesplayer
                ovr_seriesplayer.to_csv(f'!Data/Playoffs/{series}/{player.title() + series}.csv')
                continue
                
            if newplayer.columns.equals(ovrplayer.columns) and newplayer.columns.equals(ovr_seriesplayer.columns): # Make sure the new and overall dataframes can be added without error
                ovrplayer = ovrplayer.add(newplayer)
                ovr_player_data[player] = ovrplayer
                ovrplayer.to_csv(f'!Data/Playoffs/{player.title()}.csv') # Write the updated overall player file to the overall data directory
                if game == 'Game1':
                    continue
                    
                ovr_seriesplayer = ovr_seriesplayer.add(newplayer)
                ovr_series_player_data[player] = ovr_seriesplayer
                ovr_seriesplayer.to_csv(f'!Data/Playoffs/{series}/{player.title() + series}.csv' )# Write the updated overall series player file to the overall series data directory
            else:
                print(f"{player.title()}'s columns do not match")  
            continue
            
    return new_player_data, ovr_player_data, ovr_series_player_data # Return both dictionaries as a tuple

new_player_data, ovr_player_data, ovr_series_player_data  = load_data(players) # Unpack the tuple of dictionaries into their respective names

In [4]:
newMurrayStats = PPP(new_player_data['murray'])
newMurrayStats.to_csv(f'Murray/PPP Stats/{prevGames[-1] + Opp}.csv')
newMurrayStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,0.98,53.3,5,61.1,1.0,38.5,55.6,9,55.6,0,N/A,0.97,74.7,66.7,7,85.7,2,0.0
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,1.64,11.1,1,100.0,1.55,14.9,100.0,1,100.0,1,100.0,2.0,5.6,100.0,1,100.0,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,0.67,6.8,0,33.3,0.67,11.5,33.3,2,50.0,1,0.0,N/A,0.0,N/A,0,N/A,0,N/A
TRANSITION,0.34,13.4,1,0.0,0.69,11.1,0.0,1,0.0,0,N/A,0.0,16.7,0.0,0,N/A,3,0.0
Attacking Closeouts,1.5,4.5,0,50.0,3.0,3.8,100.0,0,N/A,1,100.0,0.0,5.6,0.0,0,N/A,1,0.0
Catch & Shoot,1.0,6.8,0,33.3,1.0,11.5,33.3,0,N/A,3,33.3,N/A,0.0,N/A,0,N/A,0,N/A


In [5]:
MurraySeriesStats = PPP(ovr_series_player_data['murray'])
MurraySeriesStats.to_csv(f'Murray/PPP Stats/{prevGames[-1] + Opp}.csv')
MurraySeriesStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,1.1,36.8,8,59.6,1.22,28.0,59.1,18,61.1,4,50.0,1.03,46.2,60.0,15,80.0,10,30.0
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,1.89,8.4,1,90.0,1.42,7.7,75.0,2,100.0,2,50.0,2.33,9.1,100.0,4,100.0,2,100.0
DHO Screener,0.0,1.3,0,0.0,N/A,0.0,N/A,0,N/A,0,N/A,0.0,2.6,0.0,2,0.0,0,N/A
DBL Ball Handler,0.0,1.3,0,0.0,N/A,0.0,N/A,0,N/A,0,N/A,0.0,2.6,0.0,0,N/A,2,0.0
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,0.71,14.3,1,22.2,0.57,17.1,30.8,7,57.1,6,0.0,0.94,11.2,0.0,0,N/A,5,0.0
TRANSITION,1.38,18.3,2,56.5,0.98,13.7,42.9,7,42.9,0,N/A,1.63,23.4,62.5,7,71.4,9,55.6
Attacking Closeouts,1.75,5.1,0,75.0,2.5,4.9,100.0,2,100.0,2,100.0,1.0,5.3,50.0,2,100.0,2,0.0
Catch & Shoot,1.38,8.2,0,46.2,1.38,15.9,46.2,0,N/A,13,46.2,N/A,0.0,N/A,0,N/A,0,N/A


In [6]:
murrayStats = PPP(ovr_player_data['murray'])
murrayStats.to_csv('!PPP Stats/murrayStats.csv')
murrayStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,1.24,34.5,20,55.5,1.34,32.0,58.4,94,62.8,43,48.8,1.1,38.3,51.1,45,60.0,45,42.2
PNR Screener,1.75,0.5,1,100.0,2.0,0.4,100.0,2,100.0,0,N/A,1.5,0.6,100.0,0,N/A,1,100.0
DHO Ball Handler,1.11,12.7,7,48.2,1.11,12.8,47.3,36,50.0,19,42.1,1.12,12.4,50.0,18,61.1,10,30.0
DHO Screener,0.0,0.3,0,0.0,N/A,0.0,N/A,0,N/A,0,N/A,0.0,0.6,0.0,2,0.0,0,N/A
DBL Ball Handler,0.93,1.6,0,41.7,1.02,1.0,50.0,4,50.0,0,N/A,0.88,2.6,37.5,4,50.0,4,25.0
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,1.02,20.9,13,45.9,0.83,21.1,42.0,64,53.1,24,12.5,1.32,20.6,53.2,23,73.9,24,33.3
TRANSITION,1.28,15.9,7,54.8,1.25,12.9,50.0,27,51.9,19,47.4,1.31,20.6,58.6,33,75.8,25,36.0
Attacking Closeouts,1.46,2.5,0,57.9,1.79,2.7,66.7,6,50.0,6,83.3,0.86,2.3,42.9,3,100.0,4,0.0
Catch & Shoot,1.03,4.9,0,34.2,1.05,7.8,35.1,0,N/A,37,35.1,0.0,0.3,0.0,0,N/A,1,0.0


In [7]:
newJokicStats = PPP(new_player_data['jokic'])
newJokicStats.to_csv(f'Jokic/PPP Stats/{prevGames[-1] + Opp}.csv')
newJokicStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,1.0,4.9,0,50.0,2.0,3.7,100.0,1,100.0,0,N/A,0.0,7.1,0.0,1,0.0,0,N/A
PNR Screener,1.33,14.6,0,66.7,2.0,11.1,100.0,3,100.0,0,N/A,0.67,21.4,33.3,2,50.0,1,0.0
DHO Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,2.25,11.9,0,100.0,2.0,3.7,100.0,1,100.0,0,N/A,2.32,27.7,100.0,2,100.0,1,100.0
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,0.88,38.7,2,46.2,0.54,47.7,30.0,10,30.0,0,N/A,2.33,21.4,100.0,2,100.0,1,100.0
TRANSITION,0.0,4.6,0,0.0,N/A,0.0,N/A,0,N/A,0,N/A,0.0,13.4,0.0,0,N/A,1,0.0
Attacking Closeouts,2.0,7.3,0,100.0,2.0,3.7,100.0,1,100.0,0,N/A,2.0,14.3,100.0,2,100.0,0,N/A
Catch & Shoot,1.5,4.9,0,50.0,1.5,7.4,50.0,0,N/A,2,50.0,N/A,0.0,N/A,0,N/A,0,N/A


In [8]:
JokicSeriesStats = PPP(ovr_series_player_data['jokic'])
JokicSeriesStats.to_csv(f'Jokic/PPP Stats/{prevGames[-1] + Opp}.csv')
JokicSeriesStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,0.6,6.0,2,37.5,0.86,6.2,60.0,5,60.0,0,N/A,0.0,5.5,0.0,1,0.0,2,0.0
PNR Screener,1.4,10.7,3,78.6,1.56,9.7,87.5,6,100.0,2,50.0,1.14,12.7,66.7,5,80.0,1,0.0
DHO Ball Handler,0.0,1.2,0,0.0,0.0,1.8,0.0,2,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,1.43,14.3,0,52.6,1.91,10.3,75.0,4,100.0,4,50.0,0.97,22.4,36.4,7,42.9,4,25.0
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,2.0,0.6,0,100.0,N/A,0.0,N/A,0,N/A,0,N/A,2.0,1.8,100.0,1,100.0,0,N/A
ISOLATION,1.15,41.5,9,60.0,0.91,45.2,51.2,39,53.8,2,0.0,1.82,33.9,85.7,8,100.0,6,66.7
TRANSITION,1.18,8.1,0,60.0,1.81,5.9,100.0,4,100.0,0,N/A,0.58,12.5,33.3,3,66.7,3,0.0
Attacking Closeouts,1.95,3.4,0,100.0,1.92,3.2,100.0,1,100.0,0,N/A,2.0,3.6,100.0,2,100.0,0,N/A
Catch & Shoot,2.25,2.4,0,75.0,2.25,3.6,75.0,0,N/A,4,75.0,N/A,0.0,N/A,0,N/A,0,N/A


In [9]:
jokicStats = PPP(ovr_player_data['jokic'])
jokicStats.to_csv('!PPP Stats/jokicStats.csv')
jokicStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,1.07,5.7,6,50.0,1.04,6.0,48.1,18,44.4,9,55.6,1.11,5.3,52.4,10,80.0,11,27.3
PNR Screener,1.27,9.5,12,63.0,1.36,10.8,60.4,32,65.6,16,50.0,1.14,7.9,68.0,16,87.5,9,33.3
DHO Ball Handler,0.75,0.4,1,33.3,0.0,0.5,0.0,2,0.0,0,N/A,3.0,0.2,100.0,0,N/A,1,100.0
DHO Screener,1.12,14.7,1,45.1,1.33,6.0,51.9,20,60.0,7,28.6,1.06,25.4,43.4,57,50.9,49,34.7
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,2.0,0.7,0,85.7,2.0,0.7,75.0,2,50.0,2,100.0,2.0,0.7,100.0,3,100.0,0,N/A
ISOLATION,1.14,40.3,39,54.4,1.07,42.0,51.9,169,52.1,16,50.0,1.24,38.2,57.8,66,77.3,69,39.1
TRANSITION,1.41,10.5,2,62.1,1.78,6.1,92.6,27,92.6,0,N/A,1.24,15.8,48.3,29,62.1,31,35.5
Attacking Closeouts,0.95,3.5,2,38.5,1.02,4.1,42.9,14,42.9,0,N/A,0.83,2.7,33.3,3,66.7,9,22.2
Catch & Shoot,1.32,2.5,0,44.0,1.32,4.6,44.0,0,N/A,25,44.0,N/A,0.0,N/A,0,N/A,0,N/A


In [10]:
newMPJStats = PPP(new_player_data['MPJ'])
newMPJStats.to_csv(f'MPJ/PPP Stats/{prevGames[-1] + Opp}.csv')
newMPJStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
DHO Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
ISOLATION,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
TRANSITION,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
Attacking Closeouts,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
Catch & Shoot,0.0,28.6,0,0.0,0.0,28.6,0.0,0,N/A,2,0.0,N/A,N/A,N/A,0,N/A,0,N/A


In [11]:
MPJSeriesStats = PPP(ovr_series_player_data['MPJ'])
MPJSeriesStats.to_csv(f'MPJ/PPP Stats/{prevGames[-1] + Opp}.csv')
MPJSeriesStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,0.0,2.2,0,0.0,0.0,2.5,0.0,0,N/A,1,0.0,N/A,0.0,N/A,0,N/A,0,N/A
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,0.62,21.4,2,0.0,0.0,12.5,0.0,0,N/A,3,0.0,1.29,92.8,0.0,0,N/A,2,0.0
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,0.0,2.2,0,0.0,0.0,2.5,0.0,0,N/A,1,0.0,N/A,0.0,N/A,0,N/A,0,N/A
TRANSITION,1.06,4.2,0,0.0,1.06,4.7,0.0,0,N/A,1,0.0,N/A,0.0,N/A,0,N/A,0,N/A
Attacking Closeouts,0.29,15.6,0,14.3,0.0,15.0,0.0,2,0.0,4,0.0,2.0,20.0,100.0,1,100.0,0,N/A
Catch & Shoot,0.86,31.1,0,28.6,0.86,35.0,28.6,0,N/A,14,28.6,N/A,0.0,N/A,0,N/A,0,N/A


In [12]:
MPJStats = PPP(ovr_player_data['MPJ'])
MPJStats.to_csv('!PPP Stats/MPJStats.csv')
MPJStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,0.5,2.7,0,25.0,0.29,2.8,14.3,4,25.0,3,0.0,2.0,1.9,100.0,1,100.0,0,N/A
PNR Screener,2.0,0.3,0,100.0,2.0,0.4,100.0,1,100.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,0.8,14.1,2,27.0,0.74,13.6,29.0,10,40.0,21,23.8,1.04,16.6,16.7,1,0.0,5,20.0
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,1.32,7.8,0,56.5,1.28,6.6,56.2,10,70.0,6,33.3,1.43,13.5,57.1,3,66.7,4,50.0
TRANSITION,1.2,18.8,2,44.9,1.26,14.4,41.9,12,33.3,19,47.4,1.1,40.2,50.0,6,100.0,12,25.0
Attacking Closeouts,0.78,10.2,2,34.6,0.55,9.5,21.1,9,33.3,10,10.0,1.57,13.5,71.4,6,66.7,1,100.0
Catch & Shoot,1.14,23.6,0,39.4,1.13,28.1,38.6,2,100.0,68,36.8,2.0,1.9,100.0,1,100.0,0,N/A


In [13]:
newBruceStats = PPP(new_player_data['bruce'])
newBruceStats.to_csv(f'Bruce/PPP Stats/{prevGames[-1] + Opp}.csv')
newBruceStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,0.0,42.9,1,0.0,0.0,20.0,0.0,1,0.0,0,N/A,0.0,100.0,0.0,0,N/A,1,0.0
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
TRANSITION,2.27,12.6,0,N/A,2.27,17.6,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Attacking Closeouts,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Catch & Shoot,1.0,42.9,0,33.3,1.0,60.0,33.3,0,N/A,3,33.3,N/A,0.0,N/A,0,N/A,0,N/A


In [14]:
BruceSeriesStats = PPP(ovr_series_player_data['bruce'])
BruceSeriesStats.to_csv(f'Bruce/PPP Stats/{prevGames[-1] + Opp}.csv')
BruceSeriesStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,0.79,31.1,3,45.5,0.8,26.3,50.0,8,50.0,0,N/A,0.75,57.1,33.3,0,N/A,3,33.3
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,0.0,6.7,3,N/A,0.0,5.3,N/A,0,N/A,0,N/A,0.0,14.3,N/A,0,N/A,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,0.0,6.7,2,0.0,0.0,2.6,0.0,1,0.0,0,N/A,0.0,28.6,N/A,0,N/A,0,N/A
TRANSITION,1.23,10.8,0,50.0,1.23,12.8,50.0,4,50.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Attacking Closeouts,0.0,4.4,0,0.0,0.0,5.3,0.0,2,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Catch & Shoot,1.25,26.7,0,41.7,1.25,31.6,41.7,0,N/A,12,41.7,N/A,0.0,N/A,0,N/A,0,N/A


In [15]:
bruceStats = PPP(ovr_player_data['bruce'])
bruceStats.to_csv('!PPP Stats/bruceStats.csv')
bruceStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,1.11,24.7,7,53.3,1.07,23.0,56.4,37,59.5,2,0.0,1.17,29.1,47.6,6,50.0,15,46.7
PNR Screener,1.14,0.3,0,N/A,1.14,0.4,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,0.15,4.6,5,12.5,0.25,3.9,16.7,5,20.0,1,0.0,0.0,6.3,0.0,1,0.0,1,0.0
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,2.08,2.0,1,100.0,1.06,0.9,N/A,0,N/A,0,N/A,2.58,4.9,100.0,1,100.0,2,100.0
DBL Screener,0.0,0.4,1,N/A,N/A,0.0,N/A,0,N/A,0,N/A,0.0,1.3,N/A,0,N/A,0,N/A
ISOLATION,0.54,13.0,13,27.3,0.8,6.8,27.3,9,22.2,2,50.0,0.39,29.1,27.3,5,0.0,6,50.0
TRANSITION,1.47,23.4,2,66.7,1.41,23.7,67.5,38,68.4,2,50.0,1.62,22.6,64.7,7,85.7,10,50.0
Attacking Closeouts,1.44,3.0,1,66.7,1.11,3.1,50.0,4,50.0,0,N/A,2.5,2.5,100.0,1,100.0,1,100.0
Catch & Shoot,1.17,16.3,0,39.1,1.17,22.7,39.1,0,N/A,46,39.1,N/A,0.0,N/A,0,N/A,0,N/A


In [16]:
newGordonStats = PPP(new_player_data['gordon'])
newGordonStats.to_csv(f'Gordon/PPP Stats/{prevGames[-1] + Opp}.csv')
newGordonStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,1.04,19.2,0,50.0,0.53,17.1,0.0,1,0.0,0,N/A,2.0,25.0,100.0,1,100.0,0,N/A
TRANSITION,1.29,25.9,0,66.7,1.06,17.1,100.0,1,100.0,0,N/A,1.5,50.0,50.0,0,N/A,2,50.0
Attacking Closeouts,0.0,6.7,0,0.0,0.0,9.1,0.0,1,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Catch & Shoot,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A


In [17]:
GordonSeriesStats = PPP(ovr_series_player_data['gordon'])
GordonSeriesStats.to_csv(f'Gordon/PPP Stats/{prevGames[-1] + Opp}.csv')
GordonSeriesStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,2.0,1.9,0,100.0,2.0,2.7,100.0,1,100.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
PNR Screener,2.0,1.9,0,100.0,2.0,2.7,100.0,1,100.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,0.0,3.8,0,0.0,N/A,0.0,N/A,0,N/A,0,N/A,0.0,13.3,0.0,0,N/A,2,0.0
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,1.26,22.8,0,54.5,0.89,21.3,42.9,7,42.9,0,N/A,2.0,26.7,75.0,1,100.0,3,66.7
TRANSITION,1.71,16.8,0,71.4,1.39,7.8,100.0,2,100.0,0,N/A,1.87,39.2,60.0,0,N/A,5,60.0
Attacking Closeouts,0.0,1.9,0,0.0,0.0,2.7,0.0,1,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Catch & Shoot,2.4,9.6,0,80.0,2.4,13.5,80.0,0,N/A,5,80.0,N/A,0.0,N/A,0,N/A,0,N/A


In [18]:
gordonStats = PPP(ovr_player_data['gordon'])
gordonStats.to_csv('!PPP Stats/gordonStats.csv')
gordonStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,1.13,3.1,0,50.0,1.02,3.6,42.9,7,42.9,0,N/A,1.6,1.9,100.0,1,100.0,0,N/A
PNR Screener,1.26,5.0,2,69.2,1.44,6.3,69.2,12,66.7,1,100.0,0.0,2.1,N/A,0,N/A,0,N/A
DHO Ball Handler,0.69,0.9,0,0.0,0.69,1.3,0.0,1,0.0,1,0.0,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,0.47,4.0,1,10.0,1.06,0.9,0.0,0,N/A,1,0.0,0.37,11.2,11.1,6,16.7,3,0.0
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,1.06,0.6,0,0.0,1.06,0.9,0.0,1,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,1.23,27.0,7,57.7,1.13,24.5,54.3,39,53.8,7,57.1,1.39,32.6,64.0,11,81.8,14,50.0
TRANSITION,1.27,23.0,7,60.0,1.32,17.9,69.0,28,71.4,1,0.0,1.22,34.6,50.0,8,62.5,18,44.4
Attacking Closeouts,1.31,2.2,0,50.0,1.29,1.8,33.3,2,0.0,1,100.0,1.33,3.1,66.7,2,100.0,1,0.0
Catch & Shoot,1.06,10.4,0,36.4,1.06,15.0,36.4,2,50.0,31,35.5,N/A,0.0,N/A,0,N/A,0,N/A


In [19]:
newKCPStats = PPP(new_player_data['KCP'])
newKCPStats.to_csv(f'KCP/PPP Stats/{prevGames[-1] + Opp}.csv')
newKCPStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,2.0,16.7,0,100.0,2.0,16.7,100.0,1,100.0,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
DHO Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
ISOLATION,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
TRANSITION,0.52,64.7,1,0.0,0.52,64.7,0.0,0,N/A,2,0.0,N/A,N/A,N/A,0,N/A,0,N/A
Attacking Closeouts,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
Catch & Shoot,0.0,16.7,0,0.0,0.0,16.7,0.0,0,N/A,1,0.0,N/A,N/A,N/A,0,N/A,0,N/A


In [20]:
KCPSeriesStats = PPP(ovr_series_player_data['KCP'])
KCPSeriesStats.to_csv(f'KCP/PPP Stats/{prevGames[-1] + Opp}.csv')
KCPSeriesStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,1.2,16.7,1,75.0,1.0,7.4,100.0,1,100.0,0,N/A,1.33,100.0,66.7,2,100.0,1,0.0
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,0.5,13.3,0,25.0,0.5,14.8,25.0,4,25.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,0.0,6.7,0,0.0,0.0,7.4,0.0,0,N/A,2,0.0,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
TRANSITION,0.34,19.6,1,0.0,0.34,21.8,0.0,2,0.0,2,0.0,N/A,0.0,N/A,0,N/A,0,N/A
Attacking Closeouts,2.0,3.3,0,100.0,2.0,3.7,100.0,1,100.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Catch & Shoot,1.41,35.5,0,37.5,1.41,39.4,37.5,0,N/A,8,37.5,N/A,0.0,N/A,0,N/A,0,N/A


In [21]:
KCPStats = PPP(ovr_player_data['KCP'])
KCPStats.to_csv('!PPP Stats/KCPStats.csv')
KCPStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,1.06,7.7,3,60.0,1.1,5.3,71.4,6,66.7,1,100.0,1.0,18.2,50.0,4,100.0,4,0.0
PNR Screener,0.0,0.4,1,N/A,N/A,0.0,N/A,0,N/A,0,N/A,0.0,2.3,N/A,0,N/A,0,N/A
DHO Ball Handler,1.08,12.7,2,53.8,1.09,13.1,52.2,17,58.8,6,33.3,1.02,11.1,66.7,2,50.0,1,100.0
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,0.0,0.4,1,N/A,0.0,0.5,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,0.0,0.9,0,0.0,0.0,1.1,0.0,0,N/A,2,0.0,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,0.78,7.1,1,30.8,0.77,4.1,20.0,3,0.0,2,50.0,0.79,20.2,37.5,7,42.9,1,0.0
TRANSITION,1.15,22.3,3,46.5,1.0,20.1,35.5,19,42.1,12,25.0,1.57,31.8,75.0,7,71.4,5,80.0
Attacking Closeouts,1.05,8.1,3,56.2,1.06,8.4,57.1,11,63.6,3,33.3,1.0,6.8,50.0,0,N/A,2,50.0
Catch & Shoot,1.39,28.7,0,45.3,1.36,34.8,44.4,1,100.0,62,43.5,3.0,2.3,100.0,0,N/A,1,100.0


In [22]:
newGreenStats = PPP(new_player_data['green'])
newGreenStats.to_csv(f'Green/PPP Stats/{prevGames[-1] + Opp}.csv')
newGreenStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
TRANSITION,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Attacking Closeouts,2.0,40.0,0,100.0,2.0,100.0,100.0,2,100.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Catch & Shoot,0.0,20.0,1,N/A,N/A,0.0,N/A,0,N/A,0,N/A,0.0,33.3,N/A,0,N/A,0,N/A


In [23]:
GreenSeriesStats = PPP(ovr_series_player_data['green'])
GreenSeriesStats.to_csv(f'Green/PPP Stats/{prevGames[-1] + Opp}.csv')
GreenSeriesStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,1.81,32.5,0,50.0,1.81,46.0,50.0,2,50.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
TRANSITION,2.27,10.4,0,N/A,2.27,14.7,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Attacking Closeouts,1.74,16.9,0,100.0,2.0,16.7,100.0,2,100.0,0,N/A,1.14,17.6,N/A,0,N/A,0,N/A
Catch & Shoot,2.0,17.6,1,100.0,3.0,16.7,100.0,0,N/A,2,100.0,0.0,20.0,N/A,0,N/A,0,N/A


In [24]:
greenStats = PPP(ovr_player_data['green'])
greenStats.to_csv('!PPP Stats/greenStats.csv')
greenStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
PNR Screener,1.27,7.4,1,50.0,1.36,7.3,40.0,1,0.0,4,50.0,1.0,7.7,100.0,1,100.0,0,N/A
DHO Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,0.83,5.6,0,33.3,1.0,3.7,33.3,2,0.0,1,100.0,0.67,11.5,33.3,2,50.0,1,0.0
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,1.0,1.9,0,50.0,1.0,2.5,50.0,1,100.0,1,0.0,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,1.36,21.2,1,56.2,1.33,19.6,50.0,10,50.0,0,N/A,1.45,26.5,66.7,4,100.0,2,0.0
TRANSITION,1.55,10.9,2,71.4,1.31,9.4,66.7,3,66.7,0,N/A,2.0,15.4,75.0,1,100.0,3,66.7
Attacking Closeouts,1.24,9.8,1,66.7,1.27,9.7,66.7,6,66.7,0,N/A,1.14,10.2,N/A,0,N/A,0,N/A
Catch & Shoot,0.78,21.5,1,27.3,0.82,27.2,27.3,0,N/A,22,27.3,0.0,3.8,N/A,0,N/A,0,N/A


In [25]:
newBraunStats = PPP(new_player_data['braun'])
newBraunStats.to_csv(f'Braun/PPP Stats/{prevGames[-1] + Opp}.csv')
newBraunStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,2.0,11.1,0,100.0,2.0,12.5,100.0,1,100.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,2.0,11.1,0,100.0,N/A,0.0,N/A,0,N/A,0,N/A,2.0,100.0,100.0,1,100.0,0,N/A
TRANSITION,2.0,22.2,0,100.0,2.0,25.0,100.0,2,100.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Attacking Closeouts,2.27,9.8,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,2.27,88.0,N/A,0,N/A,0,N/A
Catch & Shoot,0.0,11.1,0,0.0,0.0,12.5,0.0,0,N/A,1,0.0,N/A,0.0,N/A,0,N/A,0,N/A


In [26]:
BraunSeriesStats = PPP(ovr_series_player_data['braun'])
BraunSeriesStats.to_csv(f'Braun/PPP Stats/{prevGames[-1] + Opp}.csv')
BraunSeriesStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,0.0,4.3,0,0.0,0.0,6.7,0.0,1,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,2.0,4.3,0,100.0,2.0,6.7,100.0,1,100.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,2.0,4.3,0,100.0,N/A,0.0,N/A,0,N/A,0,N/A,2.0,12.5,100.0,1,100.0,0,N/A
TRANSITION,2.33,26.1,0,100.0,2.0,26.7,100.0,4,100.0,0,N/A,3.0,25.0,100.0,0,N/A,2,100.0
Attacking Closeouts,2.62,29.9,0,100.0,2.0,13.3,100.0,2,100.0,0,N/A,2.87,61.0,100.0,0,N/A,4,100.0
Catch & Shoot,0.0,4.3,0,0.0,0.0,6.7,0.0,0,N/A,1,0.0,N/A,0.0,N/A,0,N/A,0,N/A


In [27]:
braunStats = PPP(ovr_player_data['braun'])
braunStats.to_csv('!PPP Stats/braunStats.csv')
braunStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,0.17,8.9,0,0.0,0.0,10.9,0.0,4,0.0,1,0.0,1.14,4.4,N/A,0,N/A,0,N/A
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,2.0,3.0,0,100.0,2.0,4.3,100.0,2,100.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,1.0,6.1,1,66.7,1.0,4.3,100.0,1,100.0,0,N/A,1.0,10.0,50.0,1,100.0,1,0.0
TRANSITION,1.64,26.7,2,84.6,1.52,25.8,80.0,10,80.0,0,N/A,1.91,28.8,100.0,1,100.0,2,100.0
Attacking Closeouts,2.03,13.5,2,100.0,1.33,6.5,100.0,2,100.0,0,N/A,2.38,29.4,100.0,0,N/A,4,100.0
Catch & Shoot,0.86,10.6,0,28.6,0.86,15.2,28.6,0,N/A,7,28.6,N/A,0.0,N/A,0,N/A,0,N/A


In [28]:
newTeamStats = PPP(new_player_data['team'])
newTeamStats.to_csv(f'Team/PPP Stats/{prevGames[-1] + Opp}.csv')
newTeamStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,0.92,21.0,6,56.5,1.08,13.5,58.3,12,58.3,0,N/A,0.79,37.4,54.5,8,75.0,3,0.0
PNR Screener,1.33,4.3,0,66.7,2.0,3.1,100.0,3,100.0,0,N/A,0.67,6.8,33.3,2,50.0,1,0.0
DHO Ball Handler,1.7,4.2,1,100.0,1.64,5.1,100.0,2,100.0,1,100.0,2.0,2.3,100.0,1,100.0,0,N/A
DHO Screener,2.25,3.5,0,100.0,2.0,1.0,100.0,1,100.0,0,N/A,2.32,8.8,100.0,2,100.0,1,100.0
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,0.92,16.3,2,47.4,0.56,18.5,28.6,13,30.8,1,0.0,2.2,11.4,100.0,4,100.0,1,100.0
TRANSITION,0.82,13.1,2,33.3,1.04,12.0,50.0,4,75.0,2,0.0,0.44,15.6,16.7,0,N/A,6,16.7
Attacking Closeouts,1.69,6.3,0,75.0,1.8,5.2,80.0,4,75.0,1,100.0,1.55,8.8,66.7,2,100.0,1,0.0
Catch & Shoot,0.69,9.3,1,25.0,0.75,12.5,25.0,0,N/A,12,25.0,0.0,2.3,N/A,0,N/A,0,N/A


In [29]:
newOppStats = PPP(new_player_data['opp'])
newOppStats.to_csv(f'Opp/PPP Stats/{prevGames[-1] + Opp}.csv')
newOppStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,0.69,22.9,1,30.0,0.53,17.9,23.5,12,25.0,5,20.0,0.87,33.8,38.5,9,44.4,4,25.0
PNR Screener,0.76,5.7,0,28.6,0.76,8.3,28.6,6,33.3,1,0.0,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,1.52,8.1,0,55.6,1.5,4.2,50.0,0,N/A,4,50.0,1.53,16.4,60.0,3,33.3,2,100.0
DHO Screener,1.31,4.9,0,50.0,1.04,3.0,50.0,2,50.0,0,N/A,1.5,9.1,50.0,0,N/A,4,50.0
DBL Ball Handler,2.0,1.4,0,100.0,2.0,2.1,100.0,2,100.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,0.91,14.2,0,38.9,1.02,13.4,45.5,10,50.0,1,0.0,0.71,15.9,28.6,3,33.3,4,25.0
TRANSITION,1.48,8.3,1,71.4,1.66,7.0,75.0,4,75.0,0,N/A,1.23,11.1,66.7,3,66.7,0,N/A
Attacking Closeouts,0.92,10.9,1,33.3,0.59,10.7,14.3,6,16.7,1,0.0,1.6,11.4,60.0,2,50.0,3,66.7
Catch & Shoot,1.26,13.7,0,42.1,1.33,18.9,44.4,0,N/A,18,44.4,0.0,2.3,0.0,1,0.0,0,N/A


In [30]:
TeamSeriesStats = PPP(ovr_series_player_data['team'])
TeamSeriesStats.to_csv(f'Team/PPP Stats/{prevGames[-1] + Opp}.csv')
TeamSeriesStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,0.99,16.5,14,54.8,1.02,12.3,56.4,34,58.8,5,40.0,0.95,25.3,52.9,18,77.8,16,25.0
PNR Screener,1.43,3.5,3,80.0,1.6,3.2,88.9,7,100.0,2,50.0,1.14,3.9,66.7,5,80.0,1,0.0
DHO Ball Handler,1.07,6.0,6,50.0,0.64,5.5,35.7,9,44.4,5,20.0,1.76,7.0,75.0,4,100.0,4,50.0
DHO Screener,1.22,5.1,0,43.5,1.91,3.1,75.0,4,100.0,4,50.0,0.74,9.2,26.7,9,33.3,6,16.7
DBL Ball Handler,0.0,0.4,0,0.0,N/A,0.0,N/A,0,N/A,0,N/A,0.0,1.1,0.0,0,N/A,2,0.0
DBL Screener,0.67,0.6,0,33.3,0.0,0.5,0.0,0,N/A,2,0.0,2.0,0.6,100.0,1,100.0,0,N/A
ISOLATION,1.08,21.0,12,50.6,0.89,21.8,44.6,56,51.8,9,0.0,1.52,19.2,66.7,10,100.0,14,42.9
TRANSITION,1.38,13.1,3,58.2,1.25,10.7,57.7,23,65.2,3,0.0,1.54,18.3,58.6,10,70.0,19,52.6
Attacking Closeouts,1.51,6.3,0,63.3,1.25,5.9,52.6,13,61.5,6,33.3,1.96,7.2,81.8,5,100.0,6,66.7
Catch & Shoot,1.39,11.5,1,45.8,1.41,16.8,45.8,0,N/A,59,45.8,0.0,0.6,N/A,0,N/A,0,N/A


In [31]:
OppSeriesStats = PPP(ovr_series_player_data['opp'])
OppSeriesStats.to_csv(f'Opp/PPP Stats/{prevGames[-1] + Opp}.csv')
OppSeriesStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,1.08,22.3,6,44.3,0.72,19.7,30.3,47,27.7,19,36.8,1.66,28.0,67.5,22,63.6,18,72.2
PNR Screener,1.44,6.2,4,66.7,1.48,7.6,63.6,17,58.8,5,80.0,1.23,3.0,100.0,2,100.0,0,N/A
DHO Ball Handler,0.82,7.1,0,29.4,0.72,4.5,20.0,4,25.0,11,18.2,0.9,12.8,36.8,13,38.5,6,33.3
DHO Screener,0.87,5.3,2,37.5,0.85,3.5,41.7,12,41.7,0,N/A,0.89,9.6,33.3,3,66.7,9,22.2
DBL Ball Handler,1.71,1.3,0,71.4,2.0,0.5,100.0,2,100.0,0,N/A,1.6,3.0,60.0,1,100.0,4,50.0
DBL Screener,2.67,0.6,0,100.0,2.67,0.8,100.0,1,100.0,2,100.0,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,1.03,16.4,7,47.4,1.03,16.2,47.1,41,51.2,10,30.0,1.04,17.0,48.0,9,77.8,16,31.2
TRANSITION,1.46,5.7,2,66.7,1.5,5.2,69.2,13,69.2,0,N/A,1.37,7.1,62.5,7,71.4,1,0.0
Attacking Closeouts,1.1,10.1,4,47.7,0.96,10.1,40.7,15,46.7,12,33.3,1.41,10.3,58.8,7,85.7,10,40.0
Catch & Shoot,1.35,14.3,0,45.5,1.37,20.4,46.1,1,100.0,75,45.3,0.0,0.6,0.0,1,0.0,0,N/A


In [32]:
teamStats = PPP(ovr_player_data['team'])
teamStats.to_csv('!PPP Stats/teamStats.csv')
teamStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,1.16,14.4,32,53.1,1.19,13.1,54.6,155,56.1,50,50.0,1.1,16.8,50.8,63,65.1,67,37.3
PNR Screener,1.29,4.1,15,65.1,1.39,4.1,62.3,44,65.9,17,52.9,1.1,4.1,72.0,16,87.5,9,44.4
DHO Ball Handler,0.96,6.2,15,42.3,0.94,6.7,41.5,65,47.7,41,31.7,1.02,5.2,44.4,20,55.0,16,31.2
DHO Screener,1.07,5.1,2,43.0,1.27,1.8,48.1,19,52.6,8,37.5,1.01,11.4,41.6,58,48.3,43,32.6
DBL Ball Handler,1.07,0.6,2,50.0,0.73,0.4,50.0,4,50.0,0,N/A,1.29,1.1,50.0,5,60.0,5,40.0
DBL Screener,1.09,0.4,1,50.0,1.01,0.5,37.5,4,50.0,4,25.0,1.33,0.3,100.0,2,100.0,0,N/A
ISOLATION,1.09,24.2,69,51.1,1.02,21.6,48.6,263,51.7,52,32.7,1.19,29.2,54.9,109,70.6,106,38.7
TRANSITION,1.34,16.4,24,58.0,1.34,13.7,59.1,142,65.5,51,41.2,1.35,21.5,56.7,82,72.0,96,43.8
Attacking Closeouts,1.17,4.4,11,52.1,1.08,4.7,48.4,44,52.3,18,38.9,1.36,3.8,59.4,14,78.6,18,44.4
Catch & Shoot,1.13,10.1,1,37.9,1.13,15.2,37.5,3,66.7,266,37.2,1.25,0.4,66.7,1,100.0,2,50.0


In [33]:
oppStats = PPP(ovr_player_data['opp'])
oppStats.to_csv('!PPP Stats/oppStats.csv')
oppStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,1.01,17.3,29,44.9,0.87,14.5,38.7,107,37.4,48,41.7,1.24,25.5,55.7,47,59.6,41,51.2
PNR Screener,1.19,6.5,11,52.5,1.18,7.8,52.1,47,63.8,24,29.2,1.26,2.7,55.6,4,50.0,5,60.0
DHO Ball Handler,0.7,4.3,0,27.3,0.63,3.1,22.9,13,46.2,22,9.1,0.78,7.6,32.3,20,35.0,11,27.3
DHO Screener,0.89,3.3,3,41.7,0.87,1.8,42.9,21,42.9,0,N/A,0.91,7.6,40.7,13,69.2,14,14.3
DBL Ball Handler,1.07,1.1,0,41.2,0.73,0.6,33.3,6,33.3,0,N/A,1.26,2.7,45.5,2,100.0,9,33.3
DBL Screener,1.86,0.4,0,71.4,1.86,0.6,71.4,2,100.0,5,60.0,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,0.92,18.6,24,42.4,0.87,18.4,39.2,169,42.0,30,23.3,1.07,19.3,51.4,29,75.9,41,34.1
TRANSITION,1.28,13.3,22,64.8,1.32,12.7,68.1,97,76.3,19,26.3,1.17,14.9,57.1,32,75.0,17,23.5
Attacking Closeouts,1.05,9.2,14,49.2,1.01,9.2,48.4,68,54.4,23,30.4,1.15,9.4,51.4,15,66.7,22,40.9
Catch & Shoot,1.1,14.0,1,37.2,1.11,18.6,37.6,4,50.0,225,37.3,0.6,1.1,20.0,2,0.0,3,33.3
